
# Part 3: Insights & Recommendations — Google Play Store Apps

**Goals**

- Combine apps metadata and user reviews to derive user opinion insights using LLM-based approaches.
- Generate a synthetic dataset of users (12 users) with 60–100 installed apps each
- Propose replacement options for apps with bad ratings for each user. 
- Four recommendation strategies were considered:
  1. Category-based app Replacement/Recommendation (Naive way)
  2. Use SentenceTransformers locally to compute embeddings for app descriptions or reviews
  3. Use OpenAI/Ollama, LangChain and embeddings to generate explanation of app recommendations.
  4. **Use OpenAI/Ollama, LangChain and embeddings for App recommendation and reasoning (Best).**

---

**Notes:**
- Only best app replacement strategy (option 4) has shown in main section
- CSVs files are under `data/input` as in earlier notebooks.

In [ ]:
import os, random, pprint
import pandas as pd, numpy as np
from pathlib import Path
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
from typing import Optional, List, Callable, Union, Literal
from pydantic import BaseModel, Field
from langchain.prompts import PromptTemplate
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_openai import ChatOpenAI
from sentence_transformers import SentenceTransformer
from sklearn.neighbors import NearestNeighbors
from langchain.vectorstores import FAISS
from langchain.schema import Document

### Data preparation

Loading the data

In [2]:
# Load apps & reviews
DATA_DIR = Path("data/input")
apps = pd.read_csv(DATA_DIR / "googleplaystore.csv")
reviews = pd.read_csv(DATA_DIR / "googleplaystore_user_reviews.csv")

apps.head(5)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


- Cleaned & standardized column names.
- Merged both datasets [apps dataset (googleplaystore.csv) and reviews dataset (googleplaystore_user_reviews.csv)] into a single frame for analysis.

In [22]:
# standardize column names
apps.columns = [c.strip().lower().replace(" ", "_") for c in apps.columns]
reviews.columns = [c.strip().lower().replace(" ", "_") for c in reviews.columns]

# Basic cleaning (same as earlier notebooks)
apps['rating'] = pd.to_numeric(apps['rating'], errors='coerce')
apps['installs'] = apps['installs'].astype(str).str.replace("+","",regex=False).str.replace(",","",regex=False)
apps['installs'] = pd.to_numeric(apps['installs'], errors='coerce').fillna(0)
apps['price'] = apps['price'].astype(str).str.replace("$","",regex=False)
apps['price'] = pd.to_numeric(apps['price'], errors='coerce').fillna(0)

# normalize reviews column if present
if 'reviews' in apps.columns:
    apps['reviews'] = apps['reviews'].astype(str).str.replace("M","e6").str.replace("K","e3").str.replace(",","",regex=False)
    apps['reviews'] = pd.to_numeric(apps['reviews'], errors='coerce').fillna(0)

# reviews text column preference
text_col = None
for c in ['translated_review','review','content','text']:
    if c in reviews.columns:
        text_col = c
        break
if text_col is None:
    raise ValueError("No text column found in reviews dataset. Expected 'translated_review' or similar.")

reviews_clean = reviews.dropna(subset=[text_col]).copy().reset_index(drop=True)

# If sentiment not present, compute a quick VADER sentiment as baseline
# nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()
reviews_clean['vader_compound'] = reviews_clean[text_col].astype(str).apply(lambda x: sid.polarity_scores(x)['compound'])

def vlabel(c): 
    if c>=0.05:
        return 'positive'  
    elif  c<=-0.05:
        return 'negative'
    else:
       return 'neutral'

# Create column for vader sentiment
reviews_clean['vader_sentiment'] = reviews_clean['vader_compound'].apply(vlabel)

# Merge minimal metadata from apps
apps_subset = apps[['app','category','rating','installs','type','price']].copy()
merged = reviews_clean.merge(apps_subset, on='app', how='left')

# Visualization
print("Apps:", apps.shape, "Reviews:", reviews.shape, "Merged rows:", merged.shape)
merged.head(5)

Apps: (10841, 13) Reviews: (64295, 5) Merged rows: (74103, 12)


,app,translated_review,sentiment,sentiment_polarity,sentiment_subjectivity,vader_compound,vader_sentiment,category,rating,installs,type,price
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333,0.9531,positive,HEALTH_AND_FITNESS,4.0,500000.0,Free,0.0
1,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333,0.9531,positive,HEALTH_AND_FITNESS,4.0,500000.0,Free,0.0
2,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462,0.6597,positive,HEALTH_AND_FITNESS,4.0,500000.0,Free,0.0
3,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462,0.6597,positive,HEALTH_AND_FITNESS,4.0,500000.0,Free,0.0
4,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000,0.6249,positive,HEALTH_AND_FITNESS,4.0,500000.0,Free,0.0


### LLM-based insight extraction using LangChain

* This cell shows how you could use LangChain to extract per-app `opinion summaries` using an LLM.
* The idea: for a given app, gather a small sample of recent reviews, and ask the LLM to summarize
* Use short samples to manage cost. 

In [4]:
# Create LLM
# llm = ChatOpenAI(
#     model="gpt-4o-mini", 
#     max_retries=2, 
#     temperature=0.0
#     )
llm = ChatOllama(
    base_url="localhost:11434",
    model="llama3.1:8b",
    temperature=0.0
)

# Create prompt
template = """You are an analyst. Summarize the main praises and complaints in the following user reviews for the app '{app_name}'.
Keep it short (two bullets for praises and two bullets for complaints) and include any recurring feature requests.

Reviews:
{reviews}
"""
prompt = PromptTemplate(input_variables=['app_name','reviews'], template=template, validate_template=True)

# Create chain
llm_chain = prompt | llm

# Usage for a particular app
app = 'Facebook'
sample_reviews = merged[merged['app']==app][text_col].dropna().astype(str).tolist()[:30]
reviews_blob = '\\n---\\n'.join(sample_reviews)

# Invoke the llm
response = llm_chain.invoke({"app_name":app, "reviews": reviews_blob})

In [5]:
print(response.content)

Here are the main praises and complaints from the user reviews:

**Praises:**

* The app is great for keeping connected with friends and family (mentioned in 2 reviews)
* Recent improvements have been satisfactory, and users appreciate the exponential leaps in design development (mentioned in 2 reviews)

**Complaints:**

* The app has too many bugs and issues, including:
	+ Notifications not showing in chronological order
	+ Difficulty posting photos or albums
	+ Marketplace icon missing or hard to find
	+ Slow loading times and frequent crashes
	+ Issues with comments, such as disappearing icons or difficulty reading replies
* Users are frustrated with the constant updates that seem to introduce more problems than solutions (mentioned in 4 reviews)
* Some users miss the "Most Recent" option in the news feed and prefer chronological order (mentioned in 2 reviews)

**Recurring feature requests:**

* A warning for people with epilepsy due to flashing icons or text
* An easy-to-use delete

In [6]:
# Top apps by positive VADER-labeled reviews
top_pos = (merged[merged['vader_sentiment']=='positive']
           .groupby(['app','category','rating','installs'])
           .size().reset_index(name='positive_count')
           .sort_values('positive_count', ascending=False).head(20))

top_pos.head(10)

,app,category,rating,installs,positive_count
795,Helix Jump,GAME,4.2,1.000000e+08,1404
243,Bowmasters,GAME,4.7,5.000000e+07,1254
319,Candy Crush Saga,GAME,4.4,5.000000e+08,1008
277,"CBS Sports App - Scores, News, Stats & Watch Live",SPORTS,4.3,5.000000e+06,976
488,Duolingo: Learn Languages Free,EDUCATION,4.7,1.000000e+08,892
19,8 Ball Pool,GAME,4.5,1.000000e+08,888
311,Calorie Counter - MyFitnessPal,HEALTH_AND_FITNESS,4.6,5.000000e+07,800
112,Angry Birds Classic,GAME,4.4,1.000000e+08,730
489,Duolingo: Learn Languages Free,FAMILY,4.7,1.000000e+08,669
733,Google Photos,PHOTOGRAPHY,4.5,1.000000e+09,604


In [7]:
# Top apps by negative VADER-labeled reviews
top_neg = (merged[merged['vader_sentiment']=='negative']
           .groupby(['app','category','rating','installs'])
           .size().reset_index(name='negative_count')
           .sort_values('negative_count', ascending=False).head(20))

top_neg.head(10)

,app,category,rating,installs,negative_count
103,Angry Birds Classic,GAME,4.4,100000000.0,630
224,Bowmasters,GAME,4.7,50000000.0,582
257,"CBS Sports App - Scores, News, Stats & Watch Live",SPORTS,4.3,5000000.0,480
17,8 Ball Pool,GAME,4.5,100000000.0,402
291,Candy Crush Saga,GAME,4.4,500000000.0,360
644,Garena Free Fire,GAME,4.5,100000000.0,276
284,Calorie Counter - MyFitnessPal,HEALTH_AND_FITNESS,4.6,50000000.0,275
736,Helix Jump,GAME,4.2,100000000.0,216
483,"Expedia Hotels, Flights & Car Rental Travel Deals",TRAVEL_AND_LOCAL,4.1,10000000.0,212
586,"Fox News – Breaking News, Live Video & News Al...",NEWS_AND_MAGAZINES,4.5,10000000.0,189


In [8]:
sent_cat = (merged.groupby(['category','vader_sentiment']).size().reset_index(name='count'))
sent_cat

,category,vader_sentiment,count
0,ART_AND_DESIGN,negative,53
1,ART_AND_DESIGN,neutral,46
2,ART_AND_DESIGN,positive,283
3,AUTO_AND_VEHICLES,negative,14
4,AUTO_AND_VEHICLES,neutral,37
...,...,...,...
94,VIDEO_PLAYERS,neutral,41
95,VIDEO_PLAYERS,positive,187
96,WEATHER,negative,42
97,WEATHER,neutral,24


Sentiment distribution by category (percentage) [sorted]

In [9]:
# Group the data by 'category' and 'vader_sentiment', count the number of instances in each group
sent_cat = merged.groupby(['category','vader_sentiment']).size().reset_index(name='count')

# Calculate the percentage for each sentiment within each category
# This is done using a lambda function that divides the count by the total sum of counts for that category and multiplies by 100
sent_cat['pct'] = sent_cat.groupby('category')['count'].transform(lambda x: x / x.sum() * 100)

# Pivot the DataFrame to have categories as rows, sentiment as columns, and percentage as values
# This converts the data into a more readable format for analysis
sent_cat_pivot = sent_cat.pivot(index='category', columns='vader_sentiment', values='pct').fillna(0)

# Sort the pivot table by the 'positive' column in descending order to show the most positive sentiments first
# The .head(10) function limits the output to the top 10 categories with the highest positive sentiment
sent_cat_pivot.sort_values('positive', ascending=False).head(10)

vader_sentiment,negative,neutral,positive
category,,,
COMICS,2.500000,8.750000,88.750000
EDUCATION,8.441558,5.726092,85.832349
AUTO_AND_VEHICLES,4.844291,12.802768,82.352941
EVENTS,12.025316,8.227848,79.746835
HEALTH_AND_FITNESS,14.923384,9.060626,76.015989
FAMILY,16.192893,8.054146,75.752961
ART_AND_DESIGN,13.874346,12.041885,74.083770
PARENTING,10.769231,15.769231,73.461538
GAME,23.262745,3.853595,72.883660


### Generate synthetic users (12 users) with 60-100 installed apps each
- Created 12 synthetic users.
- Each assigned 60–100 random installed apps from the dataset.
- Used for simulation of per-user recommendations.

In [10]:
# Get lest of unique apps from the merged dataset
unique_apps = merged['app'].dropna().unique().tolist()
random.seed(42)     # Set seed for reproducibility

# Generate synthetic user data with apps
num_users = 12
users = {}
for i in range(num_users):
    uid = f'user_{i+1:02d}'
    k = random.randint(60,100)  # Randomly select between 60-100 apps for each user
    users[uid] = random.sample(unique_apps, k)  # Select k random apps from unique_apps

# Convert to DataFrame
user_rows = []
for uid, app_list in users.items():
    for app in app_list:
        user_rows.append({'user_id': uid, 'app': app})

# create DataFrame from the list of dictionaries
user_installs_df = pd.DataFrame(user_rows)

In [11]:
# Check the number of unique users and apps installed per user
print('Synthetic users generated:', user_installs_df['user_id'].nunique())
user_installs_df.groupby('user_id').size().describe()

Synthetic users generated: 12


count     12.000000
mean      86.083333
std       10.157472
min       73.000000
25%       76.500000
50%       85.500000
75%       95.500000
max      100.000000
dtype: float64

In [12]:
user_installs_df.head(5)

,user_id,app
0,user_01,Angry Birds Classic
1,user_01,95Live -SG#1 Live Streaming App
2,user_01,Google Trips - Travel Planner
3,user_01,Buscapé - Ofertas e descontos
4,user_01,Bloomberg: Market & Financial News


Identify the bad apps per user

In [13]:
# Identify bad-rated apps per user (simple threshold)
bad_apps = set(merged[merged['rating'] < 3.5]['app'].dropna().tolist())

# For each user, list bad apps they have installed
user_bad = {}
for uid in user_installs_df['user_id'].unique():
    # installed apps by an user
    inst = set(user_installs_df[user_installs_df['user_id']==uid]['app'].tolist())

    # Find list of bad Apps from the list of all apps  
    bad = sorted(list(inst.intersection(bad_apps)))

    # Append to the dictionary
    user_bad[uid] = bad

# Show counts
pd.Series({u: len(v) for u,v in user_bad.items()}).describe()

count    12.000000
mean      2.083333
std       1.378954
min       0.000000
25%       1.000000
50%       2.000000
75%       3.000000
max       5.000000
dtype: float64

## Embeddings-based Recommendations

Options:
1. Category-based app Replacement/Recommendation (Naive way)
2. Use SentenceTransformers locally to compute embeddings for app descriptions or reviews
3. Use OpenAI/Ollama, LangChain and embeddings to generate explanation of app recommendations.
4. **Use OpenAI/Ollama, LangChain and embeddings for App recommendation and reasoning (Best).**

Data ingestion to vector db for similarity search

In [14]:
# Prepare text representation for each app
app_docs = []
sample_reviews_per_app = merged.groupby("app")[text_col].apply(lambda x: " ".join(x.astype(str).tolist()[:20]))
number_rows = 100

# Create Langchain document for each app
for i, (_, row) in enumerate(merged.iterrows()):
    if i == number_rows:
        break
    app_name = str(row["app"])  # app name
    cat = str(row["category"])  # app category  
    rating = row.get("rating", "")  # app rating
    desc = f"App: {app_name}\nCategory: {cat}\nRating: {rating}\n"  # app description with app metadata
    desc += sample_reviews_per_app.get(app_name, "")
    app_doc = Document(page_content=desc, metadata={"app": app_name, "category": cat, "rating": rating}) # langchain document
    app_docs.append(app_doc)

In [15]:
# Build FAISS vectorstore with embeddings model
# embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
embeddings = OllamaEmbeddings(model="nomic-embed-text:v1.5")    # initialize

# Create vectorstore from documents using embeddings model
vectorstore = FAISS.from_documents(app_docs, embeddings)

# Create retriever from vectorstore with search parameters
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

### Option 4. Use OpenAI/Ollama, LangChain and embeddings for App recommendation and reasoning (Best).

* Create vector db by computing app-level embeddings by concatenating title + category + short sample of reviews per app. 
* Uses similarity search to retrieve apps matching semantically.
* Recommend the App and explanation based by invoking LLM chain. 
* Response is best because it is contextually aware bad app, retrieved result

In [ ]:
# Create LLM
# llm = ChatOpenAI(
#     model="gpt-4o-mini", 
#     max_retries=2, 
#     temperature=0.0
#     )
llm = ChatOllama(
    base_url="localhost:11434",
    model="llama3.1:8b",
    temperature=0.0
)

# Define structured output model
class AppRecommendation(BaseModel):
    app_name: str = Field(..., description="Name of recommended app")
    app_rating: float | None = Field(None, description="Rating of recommended app")
    app_category: str = Field(..., description="Category of recommended app")
    explanation: str = Field(..., description="Explain in 2–3 sentences why the recommended app is a better replacement option")

# Bind LLM with structured output
llm_structure_op = llm.with_structured_output(AppRecommendation)

# Define prompt
template_app_rec = """
You are an IT portfolio advisor.
The user is looking for a replacement for app: "{bad_app}" (rating {bad_rating}, category {bad_cat}).

From the following probable candidates, recommend the best replacement:
{probable_candidates}

The recommendation must be based on:
- app categories
- ratings
- typical user feedback

Return exactly one recommended app with a short explanation (2–3 sentences).
"""

prompt_app_rec = PromptTemplate(
    input_variables=["bad_app", "bad_rating", "bad_cat", "probable_candidates"],
    template=template_app_rec
)

# Build the chain
chain_recommendation = prompt_app_rec | llm_structure_op


In [ ]:
def recommend_app_langchain(uid, bad_app):
    """Recommend replacements for a bad app using LangChain, retriever and LLM"""

    # Get installed apps by the user
    installed = set(user_installs_df[user_installs_df["user_id"] == uid]["app"].tolist())

    # Retrieve results from retriever for the bad app
    results = retriever.get_relevant_documents(bad_app)

    # Handle the situation of duplicate retrieve results
    seen = set()
    unique_results = []
    for doc in results:
        cand = doc.metadata["app"]
        if cand not in seen:
            unique_results.append(doc)
            seen.add(cand)

    # Convert retriever results into candidate list
    candidates = []
    for doc in unique_results:
        cand = {
            "app": doc.metadata.get("app", "Unknown"),
            "rating": doc.metadata.get("rating", "N/A"),
            "category": doc.metadata.get("category", "Unknown")
        }
        candidates.append(cand)

    # Format candidates for the prompt
    probable_candidates = "\n".join(
        [f"- {c['app']} (rating {c['rating']}, category {c['category']})" for c in candidates]
    )

    # Get info for the bad app itself
    bad_row = merged[merged["app"] == bad_app]
    bad_rating = float(bad_row["rating"].iloc[0]) if not bad_row.empty and pd.notna(bad_row["rating"].iloc[0]) else "N/A"
    bad_cat = bad_row["category"].iloc[0] if not bad_row.empty else "Unknown"

    # Run structured recommendation chain
    recommendation = chain_recommendation.invoke({
        "bad_app": bad_app,
        "bad_rating": bad_rating,
        "bad_cat": bad_cat,
        "probable_candidates": probable_candidates
    })

    return recommendation

In [20]:
# Example Usage
max_usrs = 5
for i, usr in enumerate(list(user_bad.keys())):
    if i == max_usrs:
        break
    
    # get bad app name
    usr_bad_app = user_bad[usr][0] if user_bad[usr] else None

    print(f"\n*********{usr}*********")
    if usr_bad_app:
        # get app recommendation
        recommendations = recommend_app_langchain(usr, usr_bad_app) 
        
        print(
            "====Bad App====",
            f"app_name: {usr_bad_app}",
            f'app_rating: {merged[merged['app'] == usr_bad_app]['rating'].iloc[0]}',
            f'app_category: {merged[merged['app'] == usr_bad_app]['category'].iloc[0]}',
            "====App Recommendation=====",
            f'app_name: {recommendations.app_name}',
            f'app_rating: {recommendations.app_rating}',
            f'app_category: {recommendations.app_category}',
            f'app_explanation: {recommendations.explanation}',
            sep='\n'
        )
    else:
        print("No bad apps for this user.")

*********user_01*********
====Bad App====
app_name: Allegiant
app_rating: 3.1
app_category: TRAVEL_AND_LOCAL
====App Recommendation=====
app_name: 10 Best Foods for You
app_rating: 4.0
app_category: HEALTH_AND_FITNESS
app_explanation: Although the category doesn't match, this app has a high rating and positive user feedback. It's likely that users looking for an Allegiant replacement might appreciate its health-focused content.
*********user_02*********
====Bad App====
app_name: ConnectLine
app_rating: 3.1
app_category: MEDICAL
====App Recommendation=====
app_name: 10 Best Foods for You
app_rating: 4.0
app_category: HEALTH_AND_FITNESS
app_explanation: Although the category is not an exact match, it's close enough to consider. The higher rating and positive user feedback make this a suitable replacement.
*********user_03*********
====Bad App====
app_name: Acorn TV: World-class TV from Britain and Beyond
app_rating: 3.0
app_category: ENTERTAINMENT
====App Recommendation=====
app_name: 10

## Summary
**Recommendation**

- Identify bad apps (low ratings/negative reviews).
- Use FAISS retriever with OpenAI embeddings to find probable replacements.
- Apply filters (rating ≥ 4.0, avoid duplicates, exclude already installed apps).
- Pass candidates to LangChain LLM with structured output for:
    - Recommended app
    - Rating
    - Category
    - 2–3 sentence explanation of why it’s a better replacement.

**Business Value**

- This framework mimics IT application portfolio rationalization:
    - Identifies underperforming apps.
    - Suggests better alternatives based on semantic similarity, ratings, and user sentiment.
    - Provides human-readable rationales for decision-makers.

- Demonstrates how AI-driven insights can help reduce redundancy, improve app satisfaction, and support cost-saving decisions.

---

## **Backup**

### Option 1: Category-based app Replacement/Recommendation (Naive way)

For each bad app, recommend up to 3 apps in same category with rating >=4.0 and not installed by user.

In [ ]:
def recommend_category_based(uid, top_n=3):
    """Function to recommend apps based on category and rating criteria"""
    
    # Get the list of bad apps for a user
    installed = set(user_installs_df[user_installs_df['user_id']==uid]['app'].tolist())
    
    # Get the merged dataframe
    recs = {}
    for app_name in user_bad[uid]:
        row = merged[merged['app']==app_name]   # data of bad app
        if row.empty:
            recs[app_name] = []
            continue

        cat = row.iloc[0]['category']   # category of bad app
        candidates = merged[(merged['category']==cat) & (~merged['app'].isin(installed))].copy() # candidates in same category not installed by user
        candidates = candidates[candidates['rating'].notna() & (candidates['rating']>=4.0)] # candidates with rating >=4.0
        candidates['score'] = candidates['rating'] - (1/(1+candidates['installs'])) # score calculation based on rating and installs
        top = candidates.sort_values('score', ascending=False).head(top_n)  # top 3 candidates based on score
        recs[app_name] = top[['app','rating','installs']].to_dict(orient='records') # recommendations as list of dictionaries
    
    return recs

In [ ]:
# Example recommendations for one user
example_usr = list(user_bad.keys())[0]
example_recs = recommend_category_based(example_usr, top_n=3)
example_usr, list(example_recs.items())[:3]

### Option 2: Use SentenceTransformers locally to compute embeddings for app descriptions or reviews

It will computes app-level embeddings by concatenating title + category + short sample of reviews per app, then uses nearest neighbors.

In [ ]:
# Build a textual representation per app
app_texts = {}
sample_reviews_per_app = merged.groupby('app')[text_col].apply(lambda x: '\\n'.join(x.astype(str).tolist()[:20]))
for a in merged['app'].dropna().unique():
    cat = merged[merged['app']==a]['category'].astype(str).values[0]
    desc = f"{a} | {cat} \\n" + (sample_reviews_per_app.get(a, '') if a in sample_reviews_per_app.index else '')
    app_texts[a] = desc

# Encode texts into embeddings
model = SentenceTransformer('all-MiniLM-L6-v2') # initialize the embedding model
names = list(app_texts.keys())  # app names
texts = [app_texts[n] for n in names]   # app descriptions
embeddings = model.encode(texts, show_progress_bar=True) # encode the text using embedding model

# Fit nearest neighbors for recommendation using cosine similarity
nbrs = NearestNeighbors(n_neighbors=10, metric='cosine').fit(embeddings)

# For a given bad app, find nearest neighbor apps not installed by user and with rating >=4.0
def recommend_embeddings(uid, bad_app, top_n=3):
    
    if bad_app not in names:
        return []
    
    idx = names.index(bad_app)
    dists, inds = nbrs.kneighbors([embeddings[idx]], n_neighbors=15)
    recs = []
    installed = set(user_installs_df[user_installs_df['user_id']==uid]['app'].tolist())
    for dist,i in zip(dists[0], inds[0]):
        cand = names[i]
        if cand in installed or cand==bad_app: continue
        row = merged[merged['app']==cand]
        if row.empty: continue
        if pd.notna(row.iloc[0]['rating']) and row.iloc[0]['rating']>=4.0:
            recs.append({'app': cand, 'rating': row.iloc[0]['rating'], 'dist': float(dist)})
        if len(recs)>=top_n: break
    return recs


In [ ]:
# Usage example
recommend_embeddings('user_06', 'App vault', top_n=3)

In [ ]:
# Build per-user recommendation report (category-based) and show top 3 bad apps per user with recommendations
user_reports = {}
for uid in user_installs_df['user_id'].unique():
    # Get app based recommendation
    recs = recommend_category_based(uid, top_n=3)
    
    user_reports[uid] = {'num_installed': int(user_installs_df[user_installs_df['user_id']==uid].shape[0]),
                         'num_bad_apps': len(user_bad[uid]),
                         'recommendations': recs}

# Show a sample report for first 3 users
import json, itertools
for uid in list(itertools.islice(user_reports.keys(),3)):
    print(f"\n=== Report for {uid} ===\nInstalled apps: {user_reports[uid]['num_installed']}, Bad apps: {user_reports[uid]['num_bad_apps']}\nSample recommendations:\n")
    sample_items = list(user_reports[uid]['recommendations'].items())[:5]
    for bad_app, recs in sample_items:
        print(f"- {bad_app} -> {recs}")

### Option 3. Use app rating for app recommendation and OpenAI/Ollama, LangChain and embeddings to generate explanation.

* Create vector db by computing app-level embeddings by concatenating title + category + short sample of reviews per app. 
* Retrieve closed app using similarity search to retrieve apps matching semantically.
* Use retrieved apps rating for recommendation
* Generate explanation of recommenced app using Lnagchain and LLM. 

In [ ]:
# Create LLM
# llm = ChatOpenAI(
#     model="gpt-4o-mini", 
#     max_retries=2, 
#     temperature=0.0
#     )
llm = ChatOllama(
    base_url="localhost:11434",
    model="llama3.1:8b",
    temperature=0.0
)

# Prompt for recommendation rationale
template = """
You are an IT portfolio advisor.
The user is looking for a replacement for app: "{bad_app}" (rating {bad_rating}, category {bad_cat}).
One candidate replacement is "{cand_app}" (rating {cand_rating}, category {cand_cat}).

Based on app categories, ratings, and typical user feedback, explain in 2–3 sentences why "{cand_app}" 
is a better replacement option than "{bad_app}".
"""

prompt = PromptTemplate(
    input_variables=["bad_app","bad_rating","bad_cat","cand_app","cand_rating","cand_cat"],
    template=template
)

rationale_chain = prompt | llm

# Recommendation function with rationale explanation
def recommend_with_explanation(uid, bad_app, top_n=3):

    # Get installed apps by the user
    installed = set(user_installs_df[user_installs_df["user_id"] == uid]["app"].tolist())
    
    # Retrieve relevant chunk of app information List[langchain_doc_object]
    results = retriever.get_relevant_documents(bad_app)

    # Handle the situation of duplicate retrieve results
    seen = set()
    unique_results = []
    for doc in results:
        cand = doc.metadata["app"]
        if cand not in seen:
            unique_results.append(doc)
            seen.add(cand)

    # Get bad app rating and category   
    bad_row = merged[merged["app"] == bad_app]  # app row 
    bad_rating = float(bad_row["rating"].iloc[0]) if not bad_row.empty and pd.notna(bad_row["rating"].iloc[0]) else "N/A"
    bad_cat = bad_row["category"].iloc[0] if not bad_row.empty else "Unknown"
    
    recs = []
    for doc in unique_results:
        cand = doc.metadata["app"]  # app name

        # skip if retrieve app is bad or it is already installed
        if cand == bad_app or cand in installed:
            continue
        
        # convert rating to float
        cand_rating = doc.metadata.get("rating")    # app rating
        try:
            cand_rating_val = float(cand_rating)
        except:
            cand_rating_val = None
        
        # consider if retrieved app has higher rating
        if cand_rating_val and cand_rating_val >= 4.0:

            # Invoke the llm chain for explanation
            explanation = rationale_chain.invoke({
                'bad_app':bad_app,
                'bad_rating':bad_rating,
                'bad_cat':bad_cat,
                'cand_app':cand,
                'cand_rating':cand_rating_val,
                'cand_cat':doc.metadata["category"]
            })

            # Append to responses
            recs.append({
                "app": cand,
                "rating": cand_rating_val,
                "category": doc.metadata["category"],
                "explanation": explanation.content.strip()
            })
        
        # break if recommendation target has reached
        if len(recs) >= top_n:
            break
    return recs

In [ ]:
# Example
example_user = list(user_bad.keys())[0]
example_bad_app = user_bad[example_user][0] if user_bad[example_user] else None

if example_bad_app:
    recommendations = recommend_with_explanation(example_user, example_bad_app, top_n=3)
    print(f"Bad App: {example_bad_app}")
    for r in recommendations:
        print(f"- Replacement: {r['app']} (Rating: {r['rating']}, Category: {r['category']})")
        print(f"  Explanation: {r['explanation']}\n")
else:
    print("No bad apps for this user.")
